In [25]:
import re
import numpy as np 
from collections import Iterable
from sklearn.cluster import DBSCAN
import Levenshtein

def de_bruijn_ize(st,k):
    edges=[]
    nodes=set()
    for i in range(len(st)-k+1):
        edges.append((st[i:i+k-1],st[i+1:i+k]))
        nodes.add(st[i:i+k-1])
        nodes.add(st[i+1:i+k])
    return nodes, edges

def visualize_de_bruijn(st,k):
    nodes, edges=de_bruijn_ize(st,k)
    dot_str='digraph "DeBruijn graph" {\n'
    for node in nodes:
        dot_str+=' %s [label="%s"] ;\n' % (node, node)
    for src, dst in edges:
        dot_str+=' %s -> %s ;\n' %(src, dst)
    return dot_str+'}\n'

def translate(seq):
      
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    protein =""
    for i in range(0, len(seq)-2, 3):
        codon = seq[i:i + 3]
        protein+= table[codon]
    return protein

def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'} 
    bases = list(seq) 
    for element in bases:
        if element not in complement:
            print(element) 
        letters = [complement[base] for base in element] 
        return ''.join(letters)
def reverse_complement(seq):
    return complement(seq[::-1])


def create_kmer_dictionary(tl, k):
    dict={}
    for t in tl:
        for m in range(2):
            for j in range(0,3):
                st=translate(t[j:])
                for i in range(len(st)-k):
                    if st[i:i+k] in dict:
                        dict[st[i:i+k]]+=1
                    else:
                        dict[st[i:i+k]]=1

                    #if st[i:i+k-1] in dict:
                    #    if st[i+k] in dict[st[i:i+k-1]]:
                    #        dict[st[i:i+k-1]][st[i+k]]+=1
                    #    else:
                    #        dict[st[i:i+k-1]][st[i+k]]=1
                    #else:
                    #    dict[st[i:i+k-1]]={st[i+k]:1}
            t=reverse_complement(t)
    dict={k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}
    return dict

def read_fastq(filename):
    with open(filename) as f:
        lines = f.readlines()
    split_lines=[re.split('[^GTCA]', l) for l in lines[1::4]]
    flattened = [val for sublist in split_lines for val in sublist]
    flattened_filtered=list(filter(None, flattened))
    return flattened_filtered

def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

# recursion function
def find_path(dict, str,k,score):
    # if last letter is stop sequence, return substring and remove from all kmers from dict
    if str[-1]=='_':
        # TODO: remove instances from dict
        return str,score/(len(str)-k+1)
    # else, find all possible paths
    else:
        next={key[-1]:val for key,val in dict.items() if key.startswith(str[-k+1:])}
        return [find_path(dict, str+n,k,score+s) for n,s in next.items()]

In [19]:
# translate into all possible frames (6)
# can jump into a different frame only between reads -  no need (worry that same read different orientation will cause loop, but low chances with many reads long length)
# save k-mer graph in hash table


# recursion
# start only at methionine
# store k-mers in dictionary, go down most common path
# can use bloom filter for search

#  [ v for k,v in my_dict.items() if k.startswith('Date')]
t=read_fastq('../Data/HIV/ERR3953893.end1.fq')
dict=create_kmer_dictionary(t,10)

In [44]:
key=list(dict.keys())
val=list(dict.values())
print(len(val))
all_but1=val.index(300)
dict_clean={k:v for k,v in zip(key[0:all_but1], val[0:all_but1])}

3423642


In [51]:
class contig:
    def __init__(self, parent, kmer, abundance):
        self.parent=parent
        self.abundance=abundance
        self.string=kmer
        self.transversed=False
    def __str__(self):
        return "Str:"+str(self.string)+"|Ab:"+str(self.abundance)

In [55]:

queue=[contig(None,k, dict_clean[k]) for k in dict_clean.keys() if k.startswith('M') and '_' not in k]
queue.sort(key=lambda x: x.abundance, reverse=True)
strand=queue[0]
queue=[]
k=10
while True:
    
    next={key[-1]:val for key,val in dict.items() if key.startswith(str[-k+1:])}
    exts=[contig(strand, key, val) for key,val in dict_clean.items() if key.startswith(strand.string[-k+1:])]
    if len(exts)==1:
        strand.string+=exts[0].string[-1]
    else:
        queue.extend([exts])
        queue.sort(key=lambda x: x.abundance, reverse=True)
        strand=queue.pop()
# do one then generalize for other genes
# if 2 kmers identical except for one bp take bigger





Str:MTNNPPIPVG|Ab:114339


In [45]:

print(len(dict_clean))

start_pos={ k: dict_clean[k] for k in dict_clean.keys() if k.startswith('M') and '_' not in k}
# TODO: add edge case for start pos in first kmer
strs=[]
for k,v in start_pos.items():
    strs.append(find_path(dict_clean, k, 10,v))

strs=flatten(strs)
strs.sort(key = lambda x: x[1],reverse=True) 
print(strs)


#merge similar strings  (different base pairs, then alternative start sites)
stn=[s[0] for s in strs]
#x1,y1=np.meshgrid(stn,stn)
#veclev=np.vectorize(Levenshtein.ratio)
#compmat=veclev(x1,y1)
#compmat>0.95

# when find path need to remove all reverse complements and alternative start sites from dictionary

def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])     # extract indices
    return Levenshtein.distance(stn[i], stn[j])

X = np.arange(len(stn)).reshape(-1, 1)



34026


RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
def make_paths(dict, str,k,score):
    # if last letter is stop sequence, return substring and remove from all kmers from dict
    if str[-1]=='_':
        # TODO: remove instances from dict
        return str,score/(len(str)-k+1)
    # else, find all possible paths
    else:
        next={key[-1]:val for key,val in dict.items() if key.startswith(str[-k+1:])}

        return [find_path(dict, str+n,k,score+s) for n,s in next.items()]

# create paths and save them
# if there is a break save path and score up to break
# then 

In [15]:
clustering=DBSCAN(eps=2, metric=lev_metric, min_samples=2).fit(X)
clustering.labels_

array([-1,  0, -1,  1,  0, -1,  1, -1, -1,  2,  2,  3,  3,  4,  4, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1])